In [ ]:
pip install -U  finance-datareader

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # pandas 명시적 import 필요
import FinanceDataReader as fdr
import random
import seaborn as sns
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader


from base.base_config import BaseConfig
from base.data_process import *
from base.loss_functions import MSE_Loss, KLD_Loss

from neural_ode_config import NeuralODEConfig
from neural_ode_ffn import NeuralODEEncoder, LatentSampler
from neural_ode_back import ODEFunc, NeuralODEDecoder
from neural_ode_model import NeuralODE


# ---------------------------------------------------------
# 1. 설정 및 장치 초기화
# ---------------------------------------------------------
config =  NeuralODEConfig(batch_size=64, num_assets=1, steps = 30, num_epochs=1000)

# [안전장치] Config에 device가 없으면 여기서 직접 설정
if hasattr(config, 'device'):
    device = config.device
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device set to: {device}")


config_dict = config.to_dict()

In [ ]:
price_df = fdr_data_with_ticker('2010-01-01', '2025-12-31', 'AAPL')

price_df = clean_price_data(price_df)

ln_price, ln_price_diff = log_data(price_df)

ln_price = clean_price_data(ln_price)
ln_price_diff = clean_price_data(ln_price_diff)

In [ ]:
processor = To_TensorSet(config)
tensor_set = processor.process(ln_price_diff)

# [2] 텐서 추출 (BTD 포맷 사용)
# NeuralODE 모델은 [Batch, Time, Dim] 입력을 기대하므로 'BTD'를 가져옵니다.
train_tensor = processed_data['train']['BTD']  # Shape: [N_train, 30, 500]
test_tensor  = processed_data['test']['BTD']   # Shape: [1, 30, 500]

# [3] TensorDataset 생성
# 입력(x)과 타겟(y)이 같은 Autoencoder 구조이므로, 데이터셋에 (x, x)를 넣거나
# 단순히 (x,)만 넣어서 꺼낼 때 활용할 수 있습니다. 
# 여기서는 가장 일반적인 (x,) 형태로 만듭니다.
train_dataset = TensorDataset(train_tensor)
test_dataset  = TensorDataset(test_tensor)

# [4] DataLoader 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=config.batch_size, # Config에 설정된 배치 크기 (예: 64)
    shuffle=True,              # 학습 시 순서를 섞음 (필수)
    drop_last=True             # 마지막 배치가 사이즈보다 작으면 버림 (차원 오류 방지)
)

# 테스트 로더는 보통 셔플하지 않고, 배치 전체를 한 번에 넣거나 1개씩 넣습니다.
test_loader = DataLoader(
    test_dataset,
    batch_size=1,              # 테스트는 하나씩 혹은 통째로
    shuffle=False
)

# -------------------------------------------------------
# [검증] 잘 만들어졌는지 확인
# -------------------------------------------------------
print(f"Train Data Shape: {train_tensor.shape}")
print(f"Train Loader Batches: {len(train_loader)}")

# 배치가 모델에 잘 들어가는지 테스트
for batch in train_loader:
    x_batch = batch[0] # TensorDataset은 튜플로 반환하므로 [0]으로 꺼냄
    print(f"Input Batch Shape: {x_batch.shape}") # [64, 30, 500] 이어야 함
    break

In [ ]:
import torch.nn.functional as F

# [1] Model Initialization
# We pass the CLASSES, not instances, because NeuralODE.__init__ instantiates them.
model = NeuralODE(
    config=config, 
    encoder=NeuralODEEncoder, 
    sampler=LatentSampler, 
    ode_function=ODEFunc, 
    decoder=NeuralODEDecoder
).to(config.device)

# [2] Optimizer
# Adam is the standard choice for VAE/ODE architectures.
optimizer = optim.Adam(
    model.parameters(), 
    lr=config.learning_rate, 
    weight_decay=config.weight_decay
)

#
# [2] The Training Loop
# ------------------------------------------------------------------------------
print(f"Starting Training on {config.device}...")
print(f"Dimension Mismatch Strategy: Slicing output {config.number_of_times} -> {config.steps}")

loss_recon = MSE_Loss()
history_loss = {'loss': [], 'recon': [], 'kld': []}

for epoch in range(config.num_epochs):
    model.train()
    epoch_loss = 0
    epoch_recon = 0
    epoch_kld = 0
    
    for batch_idx, batch in enumerate(train_loader):
        # 1. Prepare Data
        # TensorDataset wraps the tensor in a tuple, so we access [0]
        x_batch = batch[0].to(config.device) 
        
        # 2. Forward Pass
        # We pass time_steps=None to let the model generate the full high-res 
        # trajectory (120 points) defined in config.times
        optimizer.zero_grad()
        x_pred, mu, logvar = model(x_batch, time_steps=None)
        
        # 3. Compute Loss
        recon_loss = loss_recon(x_pred, x_batch)
        kld_loss = KLD_Loss(mu, logvar)
        total_loss = recon_loss + (config.kld_coeff) * kld_loss

        # 4. Backward & Optimize
        total_loss.backward()
        
        # Gradient Clipping (Optional but recommended for ODEs/RNNs)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        # 5. Accumulate Metrics
        epoch_loss += total_loss.item()
        epoch_recon += recon_loss.item()
        epoch_kld += kld_loss.item()
    
    # Average over batches
    avg_loss = epoch_loss / len(train_loader)
    avg_recon = epoch_recon / len(train_loader)
    avg_kld = epoch_kld / len(train_loader)
    
    history_loss['loss'].append(avg_loss)
    history_loss['recon'].append(avg_recon)
    history_loss['kld'].append(avg_kld)
    
    # Logging (Every 10 epochs)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{config.num_epochs}] "
              f"Loss: {avg_loss:.6f} | Recon: {avg_recon:.6f} | KLD: {avg_kld:.6f}")

print("Training Complete.")